<a href="https://colab.research.google.com/github/chaymahamdii/ARECHATBOT/blob/main/Project_ARE_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!pip install -U torch
!pip install PyMuPDF # for reading PDFs with Python
!pip install tqdm # for progress bars
!pip install sentence-transformers # for embedding models
!pip install --upgrade --force-reinstall pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import requests

# Get PDF document
pdf_path = "ensi_pdf.pdf"


if not os.path.exists(pdf_path):
  print("File doesn't exist !!!")

else:
  print(f"File {pdf_path} exists.")

File ensi_pdf.pdf exists.


In [ ]:
import fitz  # PyMuPDF
import os
from tqdm.auto import tqdm
import re

def text_formatter(text: str) -> str:
    """Nettoie le texte en supprimant les sauts de ligne et les espaces inutiles."""
    cleaned_text = " ".join(text.split())  # Supprime les multiples espaces et \n
    return cleaned_text

def count_sentences(text: str) -> int:
    """Compte approximativement le nombre de phrases en utilisant une segmentation plus robuste."""

    sentences = re.split(r'(?<=[.!?]) +', text)  # Sépare les phrases sur les signes de ponctuation suivis d'un espace
    return len(sentences)

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """Lit un PDF et stocke le contenu de chaque page dans une liste."""

    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Le fichier '{pdf_path}' n'existe pas.")

    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        raise RuntimeError(f"Erreur lors de l'ouverture du PDF : {e}")

    pages_content = []

    for page_index, page_content in tqdm(enumerate(doc), total=len(doc), desc="Lecture du PDF"):
        text = page_content.get_text()
        if not text.strip():  # Ignore empty pages
            continue

        text = text_formatter(text)
        word_count = len(text.split())

        pages_content.append({
            "page_number": page_index + 1,
            "page_char_count": len(text),
            "page_word_count": word_count,
            "page_sentence_count": count_sentences(text),
            "page_token_count": len(text) / 4,  # Approximation see : https://platform.openai.com/tokenizer
            "text": text
        })

    return pages_content


In [ ]:
pages_content = open_and_read_pdf( pdf_path = pdf_path)

import random
random.sample(pages_content, k = 2)

Lecture du PDF:   0%|          | 0/6 [00:00<?, ?it/s]

[{'page_number': 6,
  'page_char_count': 1815,
  'page_word_count': 273,
  'page_sentence_count': 12,
  'page_token_count': 453.75,
  'text': 'www.association-robotique-ensi.tn Campus universitaire de la Manouba Manouba 2010 association.robotique@ensi-uma.tn +216 40 485 430 Pole media : Les membres de ce pôle sont les premiers responsables de l’image de l’association sur les différents réseaux sociaux. Le responsable de ce pôle est le directeur de communication. Il a le droit d’affecter les différentes tâches aux membres de ce pôle tout en assurant un partage équitable de travail entre eux. Pole développement commercial : Les membres de ce pôle sont les premiers responsables de la recherche de nouveaux partenaires pour notre association ainsi que les sponsors de nos évènements. Le responsable de ce pôle est le directeur commercial. Il a le droit d’affecter les différentes tâches aux membres de ce pôle tout en assurant un partage équitable de travail entre eux. Pole projets : +216 40 48

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_content)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,text
0,1,1181,166,8,295.25,www.association-robotique-ensi.tn Campus unive...
1,2,1391,179,13,347.75,Responsables compétitions. Responsables Aérona...
2,3,1690,249,9,422.50,www.association-robotique-ensi.tn Campus unive...
3,4,1473,210,9,368.25,www.association-robotique-ensi.tn Campus unive...
4,5,1543,211,10,385.75,www.association-robotique-ensi.tn Campus unive...


In [ ]:
from spacy.lang.fr import French

nlp = French()
nlp.add_pipe("sentencizer")
doc = nlp(" bojour à tous . seconde phrase . la 3eme et la dernier phrase .")
list(doc.sents)

[ bojour à tous ., seconde phrase ., la 3eme et la dernier phrase .]

In [ ]:
for page_dict in tqdm(pages_content) :
  page_dict["sentences"] = list( nlp(page_dict["text"]).sents)
  page_dict["page_sentences_count"] = len( page_dict["sentences"] )

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_content, k = 2)

[{'page_number': 1,
  'page_char_count': 1181,
  'page_word_count': 166,
  'page_sentence_count': 8,
  'page_token_count': 295.25,
  'text': 'www.association-robotique-ensi.tn Campus universitaire de la Manouba Manouba 2010 Promouvoir la culture de la robotique au sein de l’ENSI. Former ses membres dans divers domaines y compris les systèmes embarqués, l’intelligence artificielle Organiser des évènemen ts. Participer à des compétitions de robotique dans le territoire tunisien ou bien à l’étranger. Réaliser des projets bénéfiques d ans différents domaines : la robotiq ue et les systèmes embarqués, l’intel ligence artificielle, le dévelop pement web et le développement mobile … Introduction : L’association Robotique ENSI est une association à but non lucratif fondée en 2006 au sein de l’Ecole nationale des Sciences de l’Informatique (ENSI). Elle englobe plus d’une centaine de membres actifs. Ses activités ont pour vocation de : Promouvoir la culture de la robotique au sein de l’ENSI. +21

In [ ]:
df = pd.DataFrame(pages_content)
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentences_count
count,6.000000,6.00000,6.000000,6.000000,6.00000,6.000000
mean,3.500000,1515.50000,214.666667,10.166667,378.87500,10.166667
std,1.870829,223.65308,40.638241,1.940790,55.91327,1.940790
min,1.000000,1181.00000,166.000000,8.000000,295.25000,8.000000
25%,2.250000,1411.50000,186.750000,9.000000,352.87500,9.000000
50%,3.500000,1508.00000,210.500000,9.500000,377.00000,9.500000
75%,4.750000,1653.25000,239.500000,11.500000,413.31250,11.500000
max,6.000000,1815.00000,273.000000,13.000000,453.75000,13.000000
